<a href="https://colab.research.google.com/github/fid0did0/WebScraper/blob/main/borse_web_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update #> /dev/null
!apt-get install -y chromium-chromedriver
!pip install selenium

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [43.3 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [456 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [979 kB]
Hit:13 https://ppa.l

In [3]:
import sys
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.relative_locator import locate_with
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import datetime
import re

options = Options()
#options.add_argument("--disable-dev-shm-usage")
options.add_argument("--no-sandbox")
options.add_argument('--headless')

# Launch Chrome browser
driver = webdriver.Chrome(options=options)


In [4]:
def got_next_pg(driver):
  next_pg_button_xpath="/html/body/app-root/app-wrapper/div/div[2]/app-global-search-result-overview/div/app-page-bar[1]/div/div/button[6]"
  element_to_chck = driver.find_element(By.XPATH, next_pg_button_xpath)
  str_to_chck = element_to_chck.get_attribute("title")
  if str_to_chck=="Zeige Seite 2":
    return False
  else:
    return True

In [6]:

driver.get("https://www.boerse-frankfurt.de/suchergebnisse/Frankreich")

anleihen_button_xpath="/html/body/app-root/app-wrapper/div/div[2]/app-global-search-result-overview/div/app-data-menue/div/div/div/drag-scroll/div/div/button[1]"
next_pg_button_xpath="/html/body/app-root/app-wrapper/div/div[2]/app-global-search-result-overview/div/app-page-bar[1]/div/div/button[6]"

WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, anleihen_button_xpath)))

anleihen_button = driver.find_element(By.XPATH, anleihen_button_xpath)
stich_button = driver.find_element(By.XPATH, '/html/body/app-root/app-wrapper/div/div[2]/app-global-search-result-overview/div/app-data-menue/div/div/div/drag-scroll/div/div/button[2]')

anleihen_button.click()
WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, next_pg_button_xpath)))

entry_link_list = []
for page_idx in range(2):
  #print("start processing page %d" % page_idx)
  next_pg_button=driver.find_element(By.XPATH, next_pg_button_xpath)
  for k in range(1,30):
    entry_xpath='/html/body/app-root/app-wrapper/div/div[2]/app-global-search-result-overview/div/div[2]/table/tbody/tr[%d]/td[1]/a' % (k)
    entry = driver.find_element(By.XPATH, entry_xpath)
    #print(entry.get_attribute("title"))
    entry_link_list.append(entry.get_attribute("href"))

  #first_row_name=driver.find_element(By.XPATH, first_row_xpath).get_attribute("title")
  if next_pg_button.is_enabled:
    print("page %d downloaded" % page_idx)
    next_pg_button.click()
    WebDriverWait(driver, 3).until(got_next_pg)
  else:
    print("last page %d downloaded" % page_idx)


#entry_xpath='/html/body/app-root/app-wrapper/div/div[2]/app-global-search-result-overview/div/div[2]/table/tbody/tr[1]/td[1]/a'
#entry = driver.find_element(By.XPATH, entry_xpath)
#print(entry.get_attribute("title"))


page 0 downloaded
page 1 downloaded


In [122]:
for enty_link_iter in entry_link_list:
  #print(enty_link_iter)
  driver.get(enty_link_iter)
  anleihe_name_xpath = "/html/body/app-root/app-wrapper/div/div[2]/app-bond/div[1]/div/app-widget-datasheet-header/div/div/div/div/div[1]/div/h1"
  ablauf_date_xpath = "/html/body/app-root/app-wrapper/div/div[2]/app-bond/div[2]/div[5]/div[1]/app-widget-master-data-bond/div/div/table/tbody/tr[7]"
  price_xpath = "/html/body/app-root/app-wrapper/div/div[2]/app-bond/div[2]/div[2]/div[2]/div/div[1]/app-widget-price-box/div/div/table/tbody/tr[1]/td[2]"
  WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, ablauf_date_xpath)))
  WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, price_xpath)))

  anleihe_name = driver.find_element(By.XPATH, anleihe_name_xpath)
  #print(anleihe_name.text)
  anleihe_name_tok_lst=re.search(r'Frankreich, Republik (.*)%.(\d+)/(\d+)', anleihe_name.text)
  #print(anleihe_name_tok_lst.groups())
  last_year=anleihe_name_tok_lst.groups()[2]
  #print(last_year)

  curs_tab = driver.find_element(By.TAG_NAME, "app-widget-price-box")
  last_price = curs_tab.find_element(By.XPATH, "div/div/table/tbody/tr[1]/td[2]")
  #print(last_price.text)
  if last_price.text=='-':
    last_price = float(100)
    last_price_na = True
  else:
    last_price_na = False
    last_price = float(last_price.text.replace(",","."))

  interest_tab = driver.find_element(By.TAG_NAME, "app-widget-interest-rate")
  kupon = interest_tab.find_element(By.XPATH, "div/div/table/tbody/tr[1]/td[2]")
  #print(kupon.text)
  if kupon.text:
    interest_rate=float(kupon.text.replace("%", "").replace(",", "."))
  else:
    interest_rate=float(0)

  #print(interest_rate)

  master_data_tab = driver.find_element(By.TAG_NAME, "app-widget-master-data-bond")
  emission_date_str = master_data_tab.find_element(By.XPATH, "div/div/table/tbody/tr[7]/td[2]").text
  emission_date_lst = re.findall(r'\d+', emission_date_str)
  if emission_date_lst:
    emission_date_na = False
    emission_date = datetime.date(int(emission_date_lst[2]), int(emission_date_lst[1]), int(emission_date_lst[0]))
  else:
    emission_date_na = True
  current_date = datetime.date.today()
  expiry_date = emission_date.replace(year = 2000+int(last_year))
  duration=(expiry_date-current_date).days
  #print(expiry_date)
  #print(duration)

  #rendite_tab = driver.find_element(By.TAG_NAME, "app-widget-key-figures")
  #try:
  #  rendite = rendite_tab.find_element(By.XPATH, "div/div/table/tbody/tr[1]/td[2]")
  #  rendite_txt=rendite.text
  #except NoSuchElementException:
  #  rendite_txt="N/A"

  if last_price_na or emission_date_na:
    print("%-40sprice: ------\trendite:    N/A" % (anleihe_name.text))
  else:
    rendite_interest = interest_rate/(last_price/100)
    #print(rendite_interest)
    rendite_capital = (100-last_price)/(duration/365)
    #print(rendite_capital)
    rendite = rendite_interest + rendite_capital
    print("%-40sprice: %6.2f\trendite: %5.2f%%" % (anleihe_name.text, last_price, rendite))


Frankreich, Republik 0,5% 20/72         price:  36.00	rendite:  2.70%
Frankreich, Republik 1,75% 13/24        price:  97.99	rendite:  3.32%
Frankreich, Republik 4,25% 06/23        price: 100.09	rendite:  3.82%
Frankreich, Republik 1,25% 17/34        price:  82.60	rendite:  3.12%
Frankreich, Republik 1,25% 15/36        price:  79.01	rendite:  3.22%
Frankreich, Republik 0,75% 19/52        price:  51.42	rendite:  3.14%
Frankreich, Republik 0,000% 20/24       price:  98.06	rendite:  3.46%
Frankreich, Republik 2,25% 13/24        price:  99.01	rendite:  3.51%
Frankreich, Republik 2,5% 13/30         price:  97.55	rendite:  2.92%
Frankreich, Republik 4% 04/55           price: 109.25	rendite:  3.37%
Frankreich, Republik 0,000% 18/29       price:  83.54	rendite:  2.61%
Frankreich, Republik 3,25% 12/45        price: ------	rendite: N/A
Frankreich, Republik 0,000% 21/25       price:  95.14	rendite:  3.11%
Frankreich, Republik 4% 09/60           price: 110.05	rendite:  3.36%
Frankreich, Republik 6%

In [126]:
#driver.get("https://www.boerse-frankfurt.de/anleihe/fr0014001nn8-frankreich-republik-0-5-20-72")
#driver.get("https://www.boerse-frankfurt.de/anleihe/fr0014001n46-frankreich-republik-0-000-20-24")
driver.get("https://www.boerse-frankfurt.de/anleihe/fr0013257524-frankreich-republik-2-17-48")
anleihe_name_xpath = "/html/body/app-root/app-wrapper/div/div[2]/app-bond/div[1]/div/app-widget-datasheet-header/div/div/div/div/div[1]/div/h1"
WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, anleihe_name_xpath)))

anleihe_name = driver.find_element(By.XPATH, anleihe_name_xpath)
print(anleihe_name.text)
anleihe_name_tok_lst=re.search(r'Frankreich, Republik (.*)%.(\d+)/(\d+)', anleihe_name.text)
print(anleihe_name_tok_lst.groups())
last_year=anleihe_name_tok_lst.groups()[2]
#print(last_year)

curs_tab = driver.find_element(By.TAG_NAME, "app-widget-price-box")
last_price = curs_tab.find_element(By.XPATH, "div/div/table/tbody/tr[1]/td[2]")
last_price = float(last_price.text.replace(",","."))
print(last_price)

interest_tab = driver.find_element(By.TAG_NAME, "app-widget-interest-rate")
kupon = interest_tab.find_element(By.XPATH, "div/div/table/tbody/tr[1]/td[2]")
#print(kupon.text)
if kupon.text:
  interest_rate=float(kupon.text.replace("%", "").replace(",", "."))
else:
  interest_rate=float(0)
#print(interest_rate)

master_data_tab = driver.find_element(By.TAG_NAME, "app-widget-master-data-bond")
emission_date_str = master_data_tab.find_element(By.XPATH, "div/div/table/tbody/tr[7]/td[2]").text
emission_date_lst = re.findall(r'\d+', emission_date_str)
emission_date = datetime.date(int(emission_date_lst[2]), int(emission_date_lst[1]), int(emission_date_lst[0]))
current_date = datetime.date.today()
expiry_date = emission_date.replace(year = 2000+int(last_year))
duration=(expiry_date-current_date).days
print(expiry_date)
#print(duration)

#rendite_tab = driver.find_element(By.TAG_NAME, "app-widget-key-figures")
#try:
#  rendite = rendite_tab.find_element(By.XPATH, "div/div/table/tbody/tr[1]/td[2]")
#  rendite_txt=rendite.text
#except NoSuchElementException:
#  rendite_txt="N/A"

rendite_interest = interest_rate/(last_price/100)
print(rendite_interest)
rendite_capital = (100-last_price)/(duration/365)
print(rendite_capital)
rendite = rendite_interest + rendite_capital

print("%-40sprice: %0.2f\trendite: %0.2f%%" % (anleihe_name.text, last_price, rendite))


MaxRetryError: ignored

In [ ]:
driver.quit()